# MNIST handwritten digits classifier by ANN

**Importing libraries**

In [1]:
!pip install mnist

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mnist 
import torch
import torchvision
import torchvision.datasets as ds
mnist_ds = ds.MNIST('/root', train = True, download = True, transform = None)
mnist_ds = ds.MNIST('/root', train = False, download = True, transform = None)

Extracting /root/MNIST/raw/train-images-idx3-ubyte.gz to /root/MNIST/raw


Extracting /root/MNIST/raw/train-labels-idx1-ubyte.gz to /root/MNIST/raw



Extracting /root/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/MNIST/raw


Extracting /root/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


**Creating the ANN class**

In [3]:
class Handwritten_digits_classifier:

  def __init__(self, hidden_layer_numbers):
    self.train_set = {
     'train_images': mnist.train_images(), 
     'train_labels': mnist.test_labels()
    }
    self.test_set = { 
     'test_images': mnist.test_images(),
     'test_labels':mnist.test_labels()
    }

    file = open('sample_data/mnist_train_small.csv', 'r')
    file_list = file.readlines()
    file.close()
    example = file_list[0]
    label = example[0]
    example = example.split(',')
    example = np.asfarray(example)

    self.inputNodes = 784 # 28 * 28 = 784 
    self.outputNodes = 10 # we have 10 digits

    self.hidden_layer_numbers = hidden_layer_numbers
  
    self.input_layer_output = []
    example = example[1:]

    #**** 784*1
    example = example.reshape(784, 1)

    self.example = example


    #Flatten images 
    self.train_set['train_images'] = (self.train_set['train_images']/255) - 0.5
    self.test_set['test_images'] = (self.test_set['test_images']/255) - 0.5
    
    #Reshape the image arrays to make suitable for feeding to the model 
    self.train_set['train_images'] = self.train_set['train_images'].reshape(-1, 784)
    #self.test_set['test_images'] = self.test_set['test_images'].reshape(784, 1)
    
    #Take the number of hidden layers's nodes and funbctions
    self.hidden_layers_nodes = []
    self.hidden_layers_types = []
    for i in range(self.hidden_layer_numbers):
      self.hidden_layers_nodes.append(input('Please, enter the number of nodes in hidden layer' + str(i + 1) + ':'))
      self.hidden_layers_types.append(input('Please, specify its activatition:'))
    
    #Convert the entered number of hidden layers' nodes to int from string
    for i in range(0, self.hidden_layer_numbers):
      self.hidden_layers_nodes[i] = int(self.hidden_layers_nodes[i])
    
    #Initialize the first and last weights
    #****
    self.IHweights = (np.random.rand(self.hidden_layers_nodes[0], self.inputNodes) - 0.5)
    self.HOweights = (np.random.rand(self.outputNodes, self.hidden_layers_nodes[self.hidden_layer_numbers - 1]) - 0.5)

    #Initialize the hidden layers weights
    #*****
    self.inner_weights = []
    self.initialize_the_inner_weight(self.hidden_layer_numbers)
    
    #Declare the input and output of hidden layers variables 
    self.hidden_layers_inputs = []
    self.hidden_layers_outputs = {}
    
    #Forward pass for the hidden layers 
    self.forwardPass()

    #Final output by softmax function 
    #*****
    self.finaloutput = self.softmax(np.dot(self.HOweights, self.hidden_layers_inputs[-1]))
    print(self.finaloutput)

  
  def initialize_the_inner_weight(self, hidden_layer_numbers):
    #*****
    for i in range(0, self.hidden_layer_numbers-1):
      if i == 0:
        #*****
        self.inner_weights.append((np.random.rand(self.hidden_layers_nodes[1], self.hidden_layers_nodes[0]) - 0.5))
      else:
        self.inner_weights.append((np.random.rand(self.hidden_layers_nodes[i+1], self.hidden_layers_nodes[i]) - 0.5))

  def forwardPass(self):
    for i in range(0, self.hidden_layer_numbers):
      if i == 0:
        #*****
        if self.hidden_layers_types[i] == 'sigmoid':
          self.hidden_layers_inputs.append(self.sigmoid(np.dot(self.IHweights, self.example)))
        else:
          self.hidden_layers_inputs.append(self.relu(np.dot(self.IHweights, self.example)))          
      else:
        # *****
        if self.hidden_layers_types[i] == 'sigmoid':
          print(len(self.inner_weights))
          self.hidden_layers_inputs.append(self.sigmoid(np.dot(self.inner_weights[i-1], self.hidden_layers_inputs[-1])))
        else:
          self.hidden_layers_inputs.append(self.relu(np.dot(self.inner_weights[i-1], self.hidden_layers_inputs[-1])))
      
  def backwardPass(self):
    

  def sigmoid(self, z, derivative = False):
    #****
    z = np.clip( z, -500, 500 )
    if derivative:
      return (np.exp(-z)) / ((np.exp(-z) + 1) ** 2)
    return 1.0 / (1.0 + np.exp(-z))

  def relu(self, z, derivative = False):
    if derivative:
      if z == 0 or z < 0:
        z = 0 
      else:
        z = 1
        return z
    return np.maximum(0, z)
  
  def softmax(sef, z):
    exp = (np.exp(z - z.max()))
    return exp / np.sum(exp, axis=0)

  def show_test_images(self):
    for i in range(0, 10):
      image = self.test_set['test_images'][i]
      image = np.array(image, dtype='float')
      pixels = image.reshape((28, 28))
      plt.imshow(pixels)
      plt.show()

  def ANN_Classifier(self):
    pass


In [4]:
classifier = Handwritten_digits_classifier(3)

Please, enter the number of nodes in hidden layer1:2
Please, specify its activatition:relu
Please, enter the number of nodes in hidden layer2:25
Please, specify its activatition:relu
Please, enter the number of nodes in hidden layer3:52
Please, specify its activatition:sigmoid
2
[[0.01765337]
 [0.05478109]
 [0.1907704 ]
 [0.09051536]
 [0.1816161 ]
 [0.13383079]
 [0.06489206]
 [0.1758999 ]
 [0.02589149]
 [0.06414944]]


In [5]:
len(classifier.inner_weights)

2